In [11]:
import pandas as pd
from darts import TimeSeries
from darts.models import BlockRNNModel
from darts.dataprocessing.transformers import Scaler
from sklearn.preprocessing import MinMaxScaler
from darts.metrics import rmse, mae
import matplotlib.pyplot as plt



In [12]:
# 1. خواندن فایل اکسل
df = pd.read_excel("main_trans.xlsx")  # مسیر فایل را تنظیم کن
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df = df.dropna(subset=['DateTime'])

In [14]:
target_date = pd.to_datetime('1/13/2023  00:00:00 AM')
df_filtered = df[(df['DateTime'] >= target_date)]

In [15]:
# 3. بازنمونه‌گیری به بازه ساعتی
df = df.set_index('DateTime')
df_resampled = df.resample('1H').mean()

C:\Users\pishva_r\AppData\Local\Temp\ipykernel_6100\1880920363.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_resampled = df.resample('1H').mean()


In [16]:
# 4. پر کردن داده‌های خالی
df_resampled = df_resampled.interpolate(method='linear')

In [17]:
# 5. ساخت سری هدف و covariates
series = TimeSeries.from_dataframe(df_resampled, value_cols='oil temperature', freq='h')
# covariates = TimeSeries.from_dataframe(df_resampled, value_cols=['DAMA', 'ROTOOBAT'], freq='h')



In [18]:
# 6. نرمال‌سازی
scaler_series = Scaler(MinMaxScaler())
series_scaled = scaler_series.fit_transform(series)


In [19]:

# scaler_cov = Scaler(MinMaxScaler())
# covariates_scaled = scaler_cov.fit_transform(covariates)


In [20]:
# 7. تقسیم داده‌ها به آموزش و تست
train_ratio = 0.9
train_series, test_series = series_scaled.split_after(train_ratio)
# train_cov, test_cov = covariates_scaled.split_after(train_ratio)

In [22]:
# 8. ساخت مدل LSTM
model = BlockRNNModel(
    model='LSTM',
    input_chunk_length=48,
    output_chunk_length=12,
    hidden_dim=64,
    n_rnn_layers=2,
    dropout=0.2,
    n_epochs=20,
    random_state=42
)

In [23]:
# 9. آموزش مدل
# model.fit(series=train_series, past_covariates=train_cov, verbose=True)
model.fit(train_series)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 50.4 K | train
6 | fc              | Sequential       | 780    | train
-------------------------------------------------------------
51.2 K    Trainable params
0         Non-trainable params
51.2 K    Total params
0.205     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


BlockRNNModel(output_chunk_shift=0, model=LSTM, hidden_dim=64, n_rnn_layers=2, hidden_fc_sizes=None, dropout=0.2, activation=ReLU, input_chunk_length=48, output_chunk_length=12, n_epochs=20, random_state=42)

In [ ]:
# 10. پیش‌بینی چند گام آینده
pred = model.predict(n=len(test_series), series=train_series, past_covariates=covariates_scaled)


In [ ]:

# 11. بازگرداندن مقیاس اصلی
pred = scaler_series.inverse_transform(pred)
test_series = scaler_series.inverse_transform(test_series)


In [ ]:

# 12. ارزیابی
print(f"RMSE: {rmse(test_series, pred):.2f}")
print(f"MAE: {mae(test_series, pred):.2f}")


In [ ]:

# 13. رسم نمودار
df_test = test_series.pd_dataframe()
df_pred = pred.pd_dataframe()


In [ ]:

plt.figure(figsize=(10, 4))
plt.plot(df_test.index, df_test['POWER'], label='Actual')
plt.plot(df_pred.index, df_pred['POWER'], label='Predicted')
plt.legend()
plt.title("Actual vs Predicted POWER")
plt.xlabel("Time")
plt.ylabel("POWER")
plt.grid(True)
plt.tight_layout()
plt.show()